<a href="https://colab.research.google.com/github/JessicaZezzz/CNN/blob/main/DenseNet_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os

In [ ]:
#import libraries to look at the images
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
data_src="/content/drive/MyDrive/Dataset_2/"

First we load each image into a list, and save the corresponding label in another list

In [ ]:
data=[]
y=[]
for d in os.listdir(data_src):
    for file in os.listdir(data_src+d):
        data.append(Image.open(data_src+d+'/'+file))
        y.append(d)

Convert each image into a numpy array

In [ ]:
X=[]
for im in data:
    X.append(np.array(im))

In [ ]:
#Convert the list of data into an array
X=np.array(X)

In [ ]:
X.shape

(6400, 128, 128)

In [ ]:
#Divide by 255 to standardize the values between 0 and 1
X=X/255

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.utils import to_categorical

In [ ]:
#Let's count how many of each category we have
non=0
mild=0
mod=0
vm=0
for cat in y:
    if cat=="Mild_Demented":
        mild+=1
    elif cat=="Moderate_Demented":
        mod+=1
    elif cat=="Non_Demented":
        non+=1
    else:
        vm+=1
print("Non Demented: ",non)
print("Very Mild: ",vm)
print("Moderate: ",mod)
print("Mild :",mild)

Non Demented:  3200
Very Mild:  2240
Moderate:  64
Mild : 896


In [ ]:
#Now we change these values to numerical ones so that we can use to_categorical
#0 will be non, 1 will be very mild, 2 will be mild, 3 will be moderate
y_num=[]
for cat in y:
    if cat=="Mild_Demented":
        y_num.append(2)
    elif cat=="Moderate_Demented":
        y_num.append(3)
    elif cat=="Non_Demented":
        y_num.append(0)
    else:
        y_num.append(1)

In [ ]:
#convert to a categorical target as expected by keras
y=to_categorical(y_num)

In [ ]:
y.shape

(6400, 4)

In [ ]:
#Reshape to use in a neural network
X=X.reshape(6400,128,128,1)

In [ ]:
#Split into a training set and test set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(4800, 128, 128, 1) (1600, 128, 128, 1) (4800, 4) (1600, 4)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os 
import tqdm
import glob
import tensorflow 

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import grey2rgb

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.applications.vgg19 import VGG19 # VGG19
from tensorflow.keras.applications.resnet50 import ResNet50 # ResNet50
from tensorflow.keras.applications.xception import Xception # Xception
from tensorflow.keras.applications.mobilenet import MobileNet # MobileNet
from tensorflow.keras.applications.nasnet import NASNetMobile # NASNetMobile
from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet169
from tensorflow.keras.applications.densenet import DenseNet121 # DenseNet121
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.utils import to_categorical
from keras import optimizers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
from typeguard import typechecked
from typing import Optional

In [ ]:
import keras
from keras import applications
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from keras import Model
INPUT_SHAPE = (128, 128, 1) # used to define the input size to the model
n_output_units = 4
activation_fn = 'softmax'
densenet_121_model = applications.densenet.DenseNet121(include_top=False, weights=None, input_shape=INPUT_SHAPE, pooling='avg')
inputs = Input(shape=INPUT_SHAPE, name='input')
model_base = densenet_121_model(inputs)
output = Dense(units=n_output_units, activation=activation_fn)(model_base)
cnn = Model(inputs=inputs, outputs=output)
cnn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 128, 128, 1)]     0         
                                                                 
 densenet121 (Functional)    (None, 1024)              7031232   
                                                                 
 dense_2 (Dense)             (None, 4)                 4100      
                                                                 
Total params: 7,035,332
Trainable params: 6,951,684
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [ ]:
#Train the Convolutional Neural Network on the data
history=cnn.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test))

In [ ]:
#Get a prediction from the neural network
y_pred=cnn.predict(X_test)

In [ ]:
#The prediction is a sequence of 4 probabilities. We choose the label with the highest probability
y_val=[]
for y in y_pred:
    y_val.append(np.argmax(y))

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
#Convert back to labels 0-3
y_true=[]
for y in y_test:
    y_true.append(np.argmax(y))

In [ ]:
#Let's see how the predictions did
print(confusion_matrix(y_true,y_val))

In [ ]:
print(classification_report(y_true,y_val))

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy on test data: ",accuracy_score(y_true,y_val))